# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Half Moon Bay,37.4636,-122.4286,65.44,85,100,19.57,US,1651722807
1,Guerrero Negro,27.9769,-114.0611,59.72,81,0,13.47,MX,1651722810
2,Mataura,-46.1927,168.8643,66.47,69,47,2.08,NZ,1651722810
3,Mahébourg,-20.4081,57.7000,78.26,83,40,14.97,MU,1651722811
4,Yashalta,46.3395,42.2762,55.90,71,100,5.14,RU,1651722811


In [6]:
cleaned_weather_df=weather_df.dropna()
cleaned_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Half Moon Bay,37.4636,-122.4286,65.44,85,100,19.57,US,1651722807
1,Guerrero Negro,27.9769,-114.0611,59.72,81,0,13.47,MX,1651722810
2,Mataura,-46.1927,168.8643,66.47,69,47,2.08,NZ,1651722810
3,Mahébourg,-20.4081,57.7000,78.26,83,40,14.97,MU,1651722811
4,Yashalta,46.3395,42.2762,55.90,71,100,5.14,RU,1651722811
...,...,...,...,...,...,...,...,...,...
552,Koumac,-20.5667,164.2833,79.00,77,58,7.61,NC,1651723022
553,Berezovyy,51.6667,135.7000,49.91,77,100,2.39,RU,1651723022
554,Berlevåg,70.8578,29.0864,24.80,69,92,7.78,NO,1651723023
555,Teeli,51.0333,90.2333,41.32,34,100,3.02,RU,1651723023


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#Configure gmaps
gmaps.configure(api_key = g_key)

#Use the Lat and Lng as locations and Humidity as the weight
locations = cleaned_weather_df[["Lat", "Lng"]]

#storing humidity
humidity = cleaned_weather_df["Humidity"]

In [10]:
#plotting heatmap
figure = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

#creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=200, point_radius=5)

#adding the heat layer to fig
figure.add_layer(heat_layer)

#displaying the fig
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#filtering the cities with wind speed < 10mph, cloudines = 0, and max temp > 70 & < 80
Narrowcities_df = cleaned_weather_df.loc[(cleaned_weather_df["Wind Speed"] < 10) & 
                                 (cleaned_weather_df["Cloudiness"]==0) &
                                 (cleaned_weather_df["Max Temp"] > 70) &
                                 (cleaned_weather_df["Max Temp"] < 80)].dropna()
Narrowcities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,Yucca Valley,34.1142,-116.4322,74.39,17,0,1.99,US,1651722813
186,Najrān,17.4924,44.1277,72.25,21,0,1.59,SA,1651722880
347,Karatsu,33.4425,129.9697,78.69,57,0,6.17,JP,1651722779
407,Caucaia,-3.7361,-38.6531,75.15,84,0,2.30,BR,1651722965
460,Herat Province,34.5000,62.0000,71.35,38,0,1.99,AF,1651722988
475,La Palma,33.8464,-118.0467,75.90,76,0,6.91,US,1651722994
537,Wajima,37.4000,136.9000,77.68,33,0,6.91,JP,1651723017


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#creating the hotel_df
hotel_df = Narrowcities_df.loc[:, ["City", "Country", "Lat", "Lng"]]

#adding a new column named Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
54,Yucca Valley,US,34.1142,-116.4322,
186,Najrān,SA,17.4924,44.1277,
347,Karatsu,JP,33.4425,129.9697,
407,Caucaia,BR,-3.7361,-38.6531,
460,Herat Province,AF,34.5000,62.0000,
475,La Palma,US,33.8464,-118.0467,
537,Wajima,JP,37.4000,136.9000,


In [19]:
#adding the url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#pulling the city names, lats, and lngs from hotel_df
for index, row in hotel_df.iterrows():
    hotel_lat = row["Lat"]
    hotel_lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{hotel_lat}, {hotel_lng}"
    print(f"Retrieving Reuslts for Index {index} : {city_name}")
    hotel_name = requests.get(base_url, params=params).json()
    
    results = hotel_name["results"]
    
    try: 
        print(f"Closest Hotel in {city_name} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result...Skipping")
    
    print("------------------------------")
    
    
print("----------End of Search----------")

Retrieving Reuslts for Index 54 : Yucca Valley
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 186 : Najrān
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 347 : Karatsu
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 407 : Caucaia
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 460 : Herat Province
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 475 : La Palma
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 537 : Wajima
Missing field/result...Skipping
------------------------------
----------End of Search----------


In [20]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
54,Yucca Valley,US,34.1142,-116.4322,
186,Najrān,SA,17.4924,44.1277,
347,Karatsu,JP,33.4425,129.9697,
407,Caucaia,BR,-3.7361,-38.6531,
460,Herat Province,AF,34.5000,62.0000,
475,La Palma,US,33.8464,-118.0467,
537,Wajima,JP,37.4000,136.9000,


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df

,City,Country,Lat,Lng,Hotel Name
54,Yucca Valley,US,34.1142,-116.4322,
186,Najrān,SA,17.4924,44.1277,
347,Karatsu,JP,33.4425,129.9697,
407,Caucaia,BR,-3.7361,-38.6531,
460,Herat Province,AF,34.5000,62.0000,
475,La Palma,US,33.8464,-118.0467,
537,Wajima,JP,37.4000,136.9000,


In [22]:
hotel_map_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_info)
figure.add_layer(hotel_map_layer)
figure                                   
                                     

AttributeError: module 'collections' has no attribute 'Iterable'